<a href="https://colab.research.google.com/github/jacob-hansen/Multimodal-Activity-Classification/blob/main/YelpScrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from csv import writer
from bs4 import BeautifulSoup
from time import sleep
import random

In [4]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [5]:
worksheet = gc.open('A new spreadsheet').sheet1
sh = gc.create('Yelp Scraping Data')

In [71]:
citiesList = [["MA", "Somerville"]]#, ["MA", "Cambridge"]]
maxListings = 100 # takes approximatly the top 10*(maxListings//10) listings
maxReviewNum = 100 # takes approximatly the top 10*(maxReviewNum//10) reviews
tempReview = []  # temporarily stores review data
headerrow = ["Place Number", "Place Name", "Review Rating", "Text", "Place URL"]

In [7]:
worksheet = gc.open('Yelp Scraping Data').sheet1
worksheet.update('A'+str(1)+':E'+str(1), [headerrow])

{'spreadsheetId': '1-LeBNycotuCkFlnbCcxLWU9NOqNtn_XStoyaFIANSY0',
 'updatedCells': 5,
 'updatedColumns': 5,
 'updatedRange': 'Sheet1!A1:E1',
 'updatedRows': 1}

In [72]:
cityPlaces = []
for row in citiesList:
    currentState = row[0]
    for currentCity in row[1:]:
        searchURL = (
            "https://www.yelp.com/search?find_desc=activities&find_loc="
            + currentCity
            + "%2C+"
            + currentState
            + "&ns=1&sortby=review_count&start="
        )
        placeList = []
        i = 0
        while i < maxListings:
            """Pull All the objects of the right class from each page"""
            restaurantSearchPage = requests.get(searchURL+str(i))
            restaurantSearchPageSoup = BeautifulSoup(
                restaurantSearchPage.text, "html.parser")
            newData = restaurantSearchPageSoup.findAll(class_="css-1uq0cfn")
            """Pull URL extentions for objects of interest"""
            noValidData = True
            for data in newData:
                try:
                    if not data.find("a")["href"]:
                        continue
                    else:
                        placeURLEnding = data.find("a")["href"]
                        placeList.append(placeURLEnding)
                        noValidData = False
                except TypeError:
                    continue
            if noValidData: # Page has no more listings
                break
            else: # Up the count and keep searching
                i += 10
            sleep(random.uniform(0, 2)) # Wait random time 0-2 seconds to not get blocked

        cityPlaces.append(placeList)

  
        # gets URL for each restaurant listing
        

In [73]:
worksheet = gc.open('Yelp Scraping Data').sheet1
activityInc = 0
worksheetRow = 1001
for city in cityPlaces:
  for urlExt in city[11:]:
      print("Scraping "+"https://yelp.com"+urlExt)
      placeURL = "https://yelp.com" + urlExt
      placeName = ((urlExt.split("/")[-1]).split("?"))[0].replace("-", " ")
      i = 0
      while i < maxReviewNum:
          try:
              placeURL = "https://yelp.com/" + urlExt + "&start=" + str(i)
              placePage = requests.get(placeURL)
              placePageSoup = BeautifulSoup(
                  placePage.text, "html.parser")
              i += 10
              reviewsRawList = placePageSoup.findAll("div", class_=" review__09f24__oHr9V border-color--default__09f24__NPAKY",)
              print(len(reviewsRawList))
              for review in reviewsRawList:
                  reviewRating = int(review.find(class_="i-stars__09f24__foihJ").get("aria-label")[0])
                  reviewText = review.find(class_ = "raw__09f24__T4Ezm").get_text()
                  tempReview = []
                  tempReview.append(activityInc)
                  tempReview.append(placeName)
                  tempReview.append(reviewRating)
                  tempReview.append(reviewText)
                  tempReview.append(placeURL)
                  worksheet.update('A'+str(worksheetRow)+':E'+str(worksheetRow), [tempReview])
                  worksheetRow+=1
              sleep(random.uniform(0, 5))  # Wait random time 0-2 seconds to not get blocked
          except:
              sleep(60)
              continue
      activityInc += 1